### 작성일 : 19/08/07
### 목적:

In [1]:
from plotnine import *
import os, gc, datetime, time
import pandas as pd, numpy as np
from tqdm import tqdm_notebook
import warnings
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8] # setting figure size
warnings.filterwarnings(action='ignore')
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

In [2]:
train_trans = pd.read_csv("/Users/jeong/downloads/IEEE/input/train_transaction.csv")
train_idf = pd.read_csv("/Users/jeong/downloads/IEEE/input/train_identity.csv")

test_trans = pd.read_csv("/Users/jeong/downloads/IEEE/input/test_transaction.csv")
test_idf = pd.read_csv("/Users/jeong/downloads/IEEE/input/test_identity.csv")

In [5]:
# 현우님 eda하신거 그대로 퍼왔습니다.. 감사합니다~

start_date = datetime.datetime.strptime('2017.11.30', '%Y.%m.%d')
train_trans['timeblock'] = train_trans['TransactionDT'].apply(lambda x: datetime.timedelta(seconds = x) + start_date ) 
test_trans['timeblock'] = test_trans['TransactionDT'].apply(lambda x: datetime.timedelta(seconds = x) + start_date ) 

tb = train_trans['timeblock']
train_trans.drop('timeblock', 1, inplace=True)
train_trans.insert(0, 'timeblock', tb)

tb = test_trans['timeblock']
test_trans.drop('timeblock', 1, inplace=True)
test_trans.insert(0, 'timeblock', tb)

def account_start_date(val):
    if np.isnan(val) :
        return np.NaN
    else:
        days=  int( str(val).split('.')[0])
        return pd.Timedelta( str(days) +' days')

train_trans['account_start_day'] = train_trans['D8'].apply(account_start_date)
test_trans['account_start_day'] = test_trans['D8'].apply(account_start_date)
# trans['account_start_day'] = trans['D8'].apply(account_start_date)

In [6]:
temp_df = train_trans[train_trans['account_start_day'].notnull()].reset_index(drop=True)
temp_df['account_make_date'] = (temp_df['timeblock'] - temp_df['account_start_day'])
temp_df['make_year'] = temp_df['account_make_date'].dt.year
temp_df['make_month'] = temp_df['account_make_date'].dt.month
temp_df['make_day'] = temp_df['account_make_date'].dt.day

In [25]:
# temp_df_same = temp_df[(temp_df['make_year'] == 2018) & (temp_df['make_month'] == 4) & (temp_df['make_day'] == 24)  & (temp_df['card1'] == 15775) & (temp_df['card3'] == 150)].reset_index(drop=True)
# temp_df_same = temp_df[(temp_df['card1'] == 9633)].reset_index(drop=True)

In [32]:
temp_df['account_make_date2'] = temp_df['account_make_date'].dt.date
temp_df_same = temp_df[(temp_df['card1'] == 9633)].reset_index(drop=True)
ex = temp_df_same.groupby(['account_make_date2']).size().reset_index()
ex.columns = ['date', 'cnt']
ex.sort_values('cnt', ascending= False, inplace = True)
ex.head(20)

,date,cnt
238,2018-02-01,38
190,2017-12-14,34
172,2017-11-26,24
153,2017-11-04,24
183,2017-12-07,23
196,2017-12-20,21
189,2017-12-13,20
327,2018-05-09,20
205,2017-12-29,20
221,2018-01-14,20


In [54]:
temp_df_same = temp_df[(temp_df['make_year'] == 2017) & (temp_df['make_month'] == 11) & (temp_df['make_day'] == 4)  & (temp_df['card1'] == 9633) ].reset_index(drop=True)


In [55]:
temp_df_same[['timeblock','TransactionAmt','V1','V2','V3','V4','V5','V6','V7','V8','V9','isFraud']].head(100)

,timeblock,TransactionAmt,V1,V2,V3,V4,V5,V6,V7,V8,V9,isFraud
0,2017-12-19 21:54:12,19.006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2017-12-19 21:55:08,19.006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2017-12-19 21:57:22,19.006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2017-12-21 18:34:04,22.154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2017-12-21 18:34:54,22.154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,2018-01-17 21:43:16,11.194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,2018-01-20 05:00:48,13.992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,2018-02-06 04:11:03,12.205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,2018-02-13 16:55:45,15.830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,2018-03-01 05:19:16,13.534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [56]:
train_trans['V1'].isnull().sum()

279287

In [ ]:
# 17-12-14

In [14]:
ex = train_trans.groupby('card1').size().reset_index()
ex.columns = ['card1', 'cnt']
ex.sort_values('cnt', ascending = False)

,card1,cnt
5365,7919,14932
6615,9500,14162
11593,15885,10361
12616,17188,10344
10950,15066,7945
9143,12695,7091
9014,12544,6773
3886,6019,6771
1412,2803,6141
5106,7585,5334


In [59]:
train_trans['account_make_date'] = ((train_trans['timeblock'] - train_trans['account_start_day'])).dt.date

In [60]:
train_trans['same_user'] = train_trans['card1'].astype('str') + train_trans['D1'].astype('str') + train_trans['account_make_date'].astype('str')

In [63]:
ex = train_trans.groupby('same_user').agg({'isFraud':['count','mean']}).reset_index()
ex.columns = ['same_user','cnt','prop']
ex.sort_values('cnt', ascending = False, inplace = True)

ex.head(100)

,same_user,cnt,prop
186558,79190.0NaT,5451,0.006237
76021,158850.0NaT,5178,0.025879
205367,95000.0NaT,4949,0.029097
64706,150660.0NaT,4562,0.024331
96351,171880.0NaT,3576,0.013143
162043,60190.0NaT,2804,0.032454
125056,28030.0NaT,2558,0.016419
8311,106160.0NaT,2535,0.035108
180746,75850.0NaT,2371,0.033741
31854,125440.0NaT,2279,0.017990


In [64]:
ex.cnt.describe()

count    211395.000000
mean          2.793538
std          33.798722
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max        5451.000000
Name: cnt, dtype: float64

In [68]:
train_idf.shape[0] / train_trans['same_user'].nunique()

0.6822914449253766

In [69]:
train_idf.shape[0] / train_trans['card1'].nunique()

10.642145650409503

In [ ]:
train_trans['same_user'] = train_trans['card1'].astype('str') + train_trans['D1'].astype('str') + train_trans['account_make_date'].astype('str')